<a href="https://colab.research.google.com/github/HoracioSoldman/simple-etl-with-python/blob/main/ETL_process_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A simple ETL process with Python

In this notebook, we are going to implement a basic ETL process with Python, using the world happiness data for many countries.

### **The data**
We will employ two similar datasets:
1.    A dataset from [World Happiness Report 2021](https://worldhappiness.report/ed/2021/) in```csv``` format.

2.    The other dataset is originated from the [World Population Review](https://worldpopulationreview.com/country-rankings/happiest-countries-in-the-world) website, more precisely at the "Happiest Countries in the World 2021" section. We will download the one in ```json``` format.


### **The transformation**

In this notebook, we are interested in the **Happiness Score** of each country over the years in our datasets. The scores are labeled as ```Life Ladder``` in our first dataset and ```happiness2021```, ```happiness2020``` in the second one. The main process of the transformation includes:


1.   Keeping only the the country name, the life ladder index and the corresponding year from the first dataset.

2.   Extracting the similar values from the second dataset.

3.   Combining both datasets 

4.   Removing duplicated entries
(e.g If there are two rows with 'Afghanistan 2020', we only keep the first one.)




In [224]:
# import libraries
import pandas as pd
import glob
from datetime import datetime

### Global variables

In [225]:
log_file= 'log.txt'

output_file= 'output.csv'

## The ETL functions

### Extract

In [226]:
def downloader():
  
  # download the excel file from the World Happiness website
  !wget https://happiness-report.s3.amazonaws.com/2021/DataPanelWHR2021C2.xls -O HR2008-21.xls

  # download the json file originated from the World Population Review website
  !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ATQG7rzhCbYHISjKPlPFUdPFr3l1AWeR' -O WHR2020-21.json


In [227]:
# extract json file
def extractor():
  dataframes= {}

  # retrieve data from sources
  downloader()

  files= glob.glob('*.*')
  
  for f in files:
    
    if (f.endswith('.xls')):
      df= pd.read_excel(f)
      dataframes['main_df']= df
    
    elif(f.endswith('.json')):
      df= pd.read_json(f)
      dataframes['additional_df']= df
  
  return dataframes

### Transform

In [228]:
def combine_dfs(dfs):
  df=pd.concat(dfs, axis=0, ignore_index=True)
  return df

In [229]:
def transformer(dfs):
  main_df= dfs['main_df']
  additional_df= dfs['additional_df']

  # for the main dataframe only keep relevant data from the first dataset
  df0= main_df[['Country name', 'year', 'Life Ladder']]

  # for the additional dataset, save the contents of df1 in a dictionary
  df1_data= {'Country name': [], 'year': [], 'Life Ladder': []}

  for index, row in additional_df.iterrows():
    df1_data['Country name'].append(row['country'])
    df1_data['year'].append(2020)
    df1_data['Life Ladder'].append(row['happiness2020'])

    df1_data['Country name'].append(row['country'])
    df1_data['year'].append(2021)
    df1_data['Life Ladder'].append(row['happiness2021'])

  # convert the dictionary into a new dataframe df1
  df1= pd.DataFrame(df1_data)

  # combine the dataframes
  df=combine_dfs([df0, df1])

  # remove duplicated values on the combination of 'Country name' and 'year'
  df.drop_duplicates(subset=['Country name', 'year'], keep='first', inplace=True)

  # return the new dataframe
  return df


### Load

For the sake of simplicity, let us just save the combined data into a ```happiness-report.csv``` file which will can be loaded into any data repository in the future.

In [230]:
def loader(df):
  df.to_csv(output_file)

### Log

In [231]:
def logger(text):
  timestamp_format = '%H:%M:%S-%h-%d-%Y' #Hour-Minute-Second-MonthName-Day-Year
  current_time = datetime.now()

  timestamp = current_time.strftime('%d-%m-%Y %H:%M:%S:%f')
  with open(log_file, "a") as f:
      f.write(timestamp +'> '+ text +'\n') 

## Run

It is important to log all the different steps of the process by invoking the ```logger``` function.

In [233]:
logger('ETL: Starting the ETL process.')

In [234]:
logger('EXTRACT: Starting the Extraction stage.')

dataframes= extractor()

logger('EXTRACT: Completed the Extraction stage.')

--2022-01-22 15:10:12--  https://happiness-report.s3.amazonaws.com/2021/DataPanelWHR2021C2.xls
Resolving happiness-report.s3.amazonaws.com (happiness-report.s3.amazonaws.com)... 52.216.176.187
Connecting to happiness-report.s3.amazonaws.com (happiness-report.s3.amazonaws.com)|52.216.176.187|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 434688 (424K) [application/vnd.ms-excel]
Saving to: ‘HR2008-21.xls’

HR2008-21.xls       100%[===================>] 424.50K  --.-KB/s    in 0.02s   

2022-01-22 15:10:12 (18.9 MB/s) - ‘HR2008-21.xls’ saved [434688/434688]

--2022-01-22 15:10:12--  https://docs.google.com/uc?export=download&id=1ATQG7rzhCbYHISjKPlPFUdPFr3l1AWeR
Resolving docs.google.com (docs.google.com)... 142.250.65.78, 2607:f8b0:4004:836::200e
Connecting to docs.google.com (docs.google.com)|142.250.65.78|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-2g-docs.googleusercontent.com/docs/securesc/ha0ro93

In [235]:
logger('TRANSFORM: Starting the Transformation stage.')

dataframe= transformer(dataframes)

logger('TRANSFORM: Completed the Transformation stage.')

In [236]:
logger('LOAD: Starting the Loading stage.')

loader(dataframe)

logger('LOAD: Completed the Loading stage.')

In [237]:
logger('ETL: Completed the ETL process.')

In [238]:
# read the log file
with open(log_file, 'r') as f:
    for line in f:
        print(line) 
    

22-01-2022 15:10:12:247020> ETL: Starting the ETL process.

22-01-2022 15:10:12:258556> EXTRACT: Starting the Extraction stage.

22-01-2022 15:10:12:993728> EXTRACT: Completed the Extraction stage.

22-01-2022 15:10:13:002649> TRANSFORM: Starting the Transformation stage.

22-01-2022 15:10:13:030542> TRANSFORM: Completed the Transformation stage.

22-01-2022 15:10:13:039077> LOAD: Starting the Loading stage.

22-01-2022 15:10:13:054718> LOAD: Completed the Loading stage.

22-01-2022 15:10:13:062205> ETL: Completed the ETL process.

